In [2]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import torch
import numpy as np
import json


model = YOLO('best.pt')

In [3]:
def rounding_up(matrix): # заменяет все числа на 1 и 0
    """
    массив, что содержит в себе все цвета каждого пикселя преобразует числа цветов в 1 и нолики (0 - чёрный цвет, а 1 - белый цвет)
    """
    result = np.zeros_like(matrix)
    result[matrix > 128] = 1
    return result

def split_matrix(matrix, rows, cols):
    """
    разделяет список на 25 частей (то есть разделяет весь массив, картинку на матрицу 5 на 5 клеток)
    """
    row_size, col_size = rows // 5, cols // 5
    parts = []
    for i in range(5):
        for j in range(5):
            part = matrix[i * row_size : (i + 1) * row_size, j * col_size : (j + 1) * col_size]
            parts.append(part)
    return parts

def round_to_majority(parts):
    """
    округляет каждую часть списка до 1 или нолика в зависимости от того, какbх сзначений там больше, единиц или ноликов, после чего сохраняет\n
    список в виде матрицы 5 на 5, в котором есть только нули и положительные повторяющиеся цифры
    """
    rounded_parts = []
    for part in parts:
        count_ones = np.count_nonzero(part)
        count_zeros = part.size - count_ones
        if count_ones >= count_zeros:
            rounded_part = np.ones_like(part)
        else:
            rounded_part = np.zeros_like(part)
        rounded_parts.append(rounded_part)
    return rounded_parts

def combine_parts(rounded_parts):
    """
    сохраняет всё в виде матрицы 5 на 5, где каждая клетка равна либо положительному числу, либо нулю
    """
    combined_matrix = np.zeros((5, 5))
    for i in range(5):
        for j in range(5):
            part_index = i * 5 + j
            combined_matrix[i, j] = np.sum(rounded_parts[part_index])
    return combined_matrix

def final_rounding(matrix):
     """
     преобразуем наш конечный список из 25 чисел в 0 и единицы, а именно исправляем баг, где числы не равные 0 могут быть больше 1, например 4260 и т.п.
     """
     matrix[matrix > 0] = 1
     return matrix

def read_from_json(filename):
    # Открытие файла для чтения
    with open(filename, 'r') as json_file:
        # Загрузка данных из файла
        data = json.load(json_file)
    return data

def load_json_to_matrices(filename):
    """
    загрузка базы данных в список matrices и преобразование его в словарь numpy
    """
    with open(filename, 'r') as json_file:
        data = json.load(json_file)

    matrices = {}
    for key, value in data.items():
        matrices[key] = np.array(value)
    
    return matrices

def compare_matrices(matrix1, matrices_from_json):
    """
    сравнение нашей матрицы с матрицами из базы данных
    """
    for key, matrix2 in matrices_from_json.items():
        if np.array_equal(matrix1, matrix2):
            return key
    return None


def mainer(picture, json_file): 
    """
    Финальная функция, что содержит в себе все предыдущие. main возвращает номер карточки и вариант её ответа, если же данная картачка не
    совпадает ни с одной из карточек из базы данных, то возвращается None. (Принимает на вход саму картинку и файл с базой данных)
    """
    cb_img = picture
    cb_img_np = np.array(cb_img)
    _, cb_img_np = cv2.threshold(cb_img_np, 127, 255, 0)
    lines, columns = cb_img.shape
    the_converted_image = final_rounding(combine_parts(round_to_majority(split_matrix(rounding_up(cb_img_np), lines, columns))))
    return compare_matrices(the_converted_image, load_json_to_matrices(json_file))


In [4]:
#Трансформация карточек
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect


def four_point_transform(image, pts):

    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [5]:
#поиск "крайних точек" карточки
def find_largest_contour(image):
    # Переводим изображение в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Применяем пороговую обработку для получения бинарного изображения
    _, thresh = cv2.threshold(gray, 127, 255, 0)
    # Инвертируем бинарное изображение
    thresh = cv2.bitwise_not(thresh)
    # Находим контуры на инвертированном бинарном изображении
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Находим наибольший контур
    largest_contour = max(contours, key=cv2.contourArea)
    extreme_points = [(0, 0), (0, image.shape[0]), (image.shape[1], image.shape[0]),
                      (image.shape[1], 0)]  # Изначально углы изображения
    min_distances = [np.inf, np.inf, np.inf, np.inf]  # Изначально расстояния до углов
    # Перебор всех точек контура для поиска крайних
    for point in largest_contour:
        x, y = point[0][0], point[0][1]
        distances = [x, image.shape[0] - y, image.shape[1] - x, y]
        for i in range(4):
            if distances[i] < min_distances[i]:
                min_distances[i] = distances[i]
                extreme_points[i] = (x, y)
    return extreme_points

In [6]:
#функция вырезки карточек из изображения
def crop_images(img, coordinates):
    images = []
    # Применяем пороговую обработку для получения бинарного изображения
    for coords in coordinates:
        left, top, right, bottom = map(int, coords)
        cropped_img = img[top:bottom, left:right]
        images.append(cropped_img)
    return images

In [7]:
#функция поворота карточи на угол angle
def rotate_image(image, angle):
    # Получаем высоту и ширину изображения
    height, width = image.shape[:2]
    # Вычисляем центр поворота
    center = (width / 2, height / 2)
    # Поворачиваем изображение на заданный угол
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


In [8]:
#----------------------------------------------------------------

In [12]:
#Получение изображения с карточками в ограничивающей рамке
def get_processed_photo(photo):
    photo=cv2.imread(photo)
    results=model(photo,conf=0.8)
    im_array = results[0].plot()
    return im_array
result_photo=get_processed_photo('122.jpg')
cv2.imshow('result',result_photo)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 640x640 17 cards, 52.0ms
Speed: 8.0ms preprocess, 52.0ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)


In [29]:
#Сохранение вырезанных карточек без ориентации
def save_crop_photos(img,save_dir):
    results=model(img,conf=0.8)
    results[0].save_crop(save_dir)
save_crop_photos(cv2.imread('100.jpg'),'C:/Users/Admin/Downloads/dfdsf/save_img')

In [8]:
#Вывод варинтов ответа по карточкам из фотографии
def get_results_photo(img):
    answers=[]
    results=model(img)
    boxes = results[0].boxes.xyxy.cpu().tolist()
    images=crop_images(img,boxes)
    for i in range(len(images)):
        image=images[i]
        image = cv2.copyMakeBorder(image, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=[255, 255, 255])
        image=rotate_image(image,30)
        pts = np.array(find_largest_contour(image), dtype = "float32")
        warped = four_point_transform(image, pts)
        gray_image = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        count=0
        while mainer(gray_image,'image_database.json')==None and count<=2:
            image=rotate_image(image,-60)
            pts = np.array(find_largest_contour(image), dtype = "float32")
            warped = four_point_transform(image, pts)
            gray_image = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
            count+=1
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        #cv2.imshow("Original", image)
        #cv2.imshow("Warped", gray_image)
        #cv2.waitKey(0)
        answers.append(mainer(gray_image,'image_database.json'))
    return answers
print(get_results_photo(cv2.imread('96.jpg')))
#96-идеал
#113-Ошибка
#109-ошибка
#107-ошибка
#100-ошибка



0: 384x640 8 cards, 240.0ms
Speed: 6.0ms preprocess, 240.0ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)
['16C', '5C', '10D', '5D', '9D', '4D', '21D', '17D']


In [27]:
#демонстрация работы модели на видео
def real_time_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 inference on the frame
            results = model(frame,verbose=False,conf=0.8)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Display the annotated frame
            cv2.imshow("YOLOv8 Inference", annotated_frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
    cap.release()
    cv2.destroyAllWindows()

real_time_video('My-video-Дата.mp4')

In [24]:
#демонстрация работы модели + алгоритма распознования карточек на видео
def video_detect_card(video_path):
    frame_counter=0
    cap = cv2.VideoCapture(video_path)
    answers = set()
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 inference on the frame
            results = model(frame,verbose=False,conf=0.8)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            frame_counter += 1
            # Display the annotated frame
            cv2.imshow("YOLOv8 Inference", annotated_frame)
            if frame_counter % 3 == 0:
                result = get_results_photo(annotated_frame)
                for value in result:
                        answers.add(value)
            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
    cap.release()
    cv2.destroyAllWindows()
    return(answers)

print(video_detect_card('My-video-Дата.mp4'))

{None, '2A', '1D', '1B', '1A', '2D', '2C', '1C', '2B'}



0: 640x640 17 cards, 45.0ms
Speed: 784.0ms preprocess, 45.0ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)


In [20]:
#Демонстрация работы нейросети на видео в реальном времени
def Real_time_demonstration():
    # Захватываем видеопоток с первой доступной веб-камеры
    cap = cv2.VideoCapture(0)

    while True:
        # Считываем кадр с веб-камеры
        ret, frame = cap.read()
        results = model(frame,verbose=False,conf=0.8)
        cv2.imshow('Webcam', results[0].plot())
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Освобождаем ресурсы
    cap.release()
    cv2.destroyAllWindows()

# Запускаем функцию для вывода видео с веб-камеры
Real_time_demonstration()

In [12]:
#Демонстрация работы нейросети + алгоритма распознования карточек в реальном времени
def Real_time_define():
    # Захватываем видеопоток с первой доступной веб-камеры
    cap = cv2.VideoCapture(0)
    frame_counter=0
    answers=set()
    while True:
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 inference on the frame
            results = model(frame,verbose=False,conf=0.8)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            frame_counter += 1
            # Display the annotated frame
            cv2.imshow("YOLOv8 Inference", annotated_frame)
            if frame_counter % 3 == 0:
                result = get_results_photo(annotated_frame)
                for value in result:
                        answers.add(value)
            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

    # Освобождаем ресурсы
    cap.release()
    cv2.destroyAllWindows()
    return answers
print(Real_time_define())

{None, '3C', '1D', '1B', '1A', '2C', '3D'}


In [15]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import torch
import numpy as np
import json
from image_recognition import *

model = YOLO('best.pt')


# Трансформация карточек
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect


def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped


# поиск "крайних точек" карточки
def find_largest_contour(image):
    # Переводим изображение в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Применяем пороговую обработку для получения бинарного изображения
    _, thresh = cv2.threshold(gray, 127, 255, 0)
    # Инвертируем бинарное изображение
    thresh = cv2.bitwise_not(thresh)
    # Находим контуры на инвертированном бинарном изображении
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Находим наибольший контур
    largest_contour = max(contours, key=cv2.contourArea)
    extreme_points = [(0, 0), (0, image.shape[0]), (image.shape[1], image.shape[0]),
                      (image.shape[1], 0)]  # Изначально углы изображения
    min_distances = [np.inf, np.inf, np.inf, np.inf]  # Изначально расстояния до углов
    # Перебор всех точек контура для поиска крайних
    for point in largest_contour:
        x, y = point[0][0], point[0][1]
        distances = [x, image.shape[0] - y, image.shape[1] - x, y]
        for i in range(4):
            if distances[i] < min_distances[i]:
                min_distances[i] = distances[i]
                extreme_points[i] = (x, y)
    return extreme_points


# функция вырезки карточек из изображения
def crop_images(img, coordinates):
    images = []
    # Применяем пороговую обработку для получения бинарного изображения
    for coords in coordinates:
        left, top, right, bottom = map(int, coords)
        cropped_img = img[top:bottom, left:right]
        images.append(cropped_img)
    return images


# функция поворота карточи на угол angle
def rotate_image(image, angle):
    # Получаем высоту и ширину изображения
    height, width = image.shape[:2]
    # Вычисляем центр поворота
    center = (width / 2, height / 2)
    # Поворачиваем изображение на заданный угол
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


def get_results_photo(img):
    answers = []
    results = model(img)
    boxes = results[0].boxes.xyxy.cpu().tolist()
    images = crop_images(img, boxes)
    for i in range(len(images)):
        image = images[i]
        image = cv2.copyMakeBorder(image, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=[255, 255, 255])
        image = rotate_image(image, 30)
        pts = np.array(find_largest_contour(image), dtype="float32")
        warped = four_point_transform(image, pts)
        gray_image = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        count = 0
        while mainer(gray_image, 'image_database.json') == None and count <= 4:
            image = rotate_image(image, -15)
            pts = np.array(find_largest_contour(image), dtype="float32")
            warped = four_point_transform(image, pts)
            gray_image = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
            count += 1
        answers.append(mainer(gray_image, 'image_database.json'))
    return answers
print(get_results_photo(cv2.imread('photo_5314671147608038937_y.jpg')))


0: 384x640 8 cards, 33.0ms
Speed: 4.0ms preprocess, 33.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
['16C', '23C', '10D', None, '4D', None, None, '17D']
